In [1]:
import cv2
import time
import imutils 
from collections import deque

import tensorflow as tf
import os
import shutil
import numpy as np
import random
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from pathlib import Path
import re
import cv2
from keras import backend as K
tf.__version__
import glob
import imageio

#check if tensorflow gpu is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version:", tf.__version__)

Num GPUs Available:  1
tensorflow version: 2.10.0


In [2]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [3]:
def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        #Debugging statements
        # if area > 1:
        #     print("Area of contour is: {}".format(area))
        
        areaMin = 60
        areaMax = 250
        if area > areaMin and area < areaMax:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            # print(len(approx))
            x , y , w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5)

            #compute center of contour
            # M = cv2.moments(cnt)
        
            # if M["m00"] != 0:
            #     cX = int(M["m10"] / M["m00"])
            #     cY = int(M["m01"] / M["m00"])
            # else:
            # # set values as what you need in the situation
            #     cX, cY = 0, 0

	        # # draw the contour and center of the shape on the image
            # cv2.circle(imgContour, (cX, cY), 7, (255, 255, 255), -1)


In [17]:
#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'
NeedleViz_oilAndLatex = 'Data/edited data/oil and latex/capture_5_2022-11-12T16-56-03.mp4'
NeedleViz_gelAndLatex = 'Data/edited data/ultrasound gel and latex/capture_4_2022-11-12T17-33-19.mp4'

#control playback speed
frame_rate = 30

#trajectory path length
trajectory_number = 3
pts = deque(maxlen=trajectory_number)

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_oilAndLatex)

fgbg = cv2.createBackgroundSubtractorMOG2() #pretty good: (100,200)
knn = cv2.createBackgroundSubtractorKNN(detectShadows=False)

# prev_frame_time = 0
# new_frame_time = 0
# average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

size = (frameWidth, frameHeight)

#Preparing to create output videos
# result = cv2.VideoWriter('Outputs/V1_videoUpdated.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, size)
image_lst = []


if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
        #Insert Code Here
        


        #Initial Frame preprocessing
        resized_frame = cv2.resize(frame, (frameWidth,frameHeight))
        resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2GRAY)


        #Achieving desired region of interest within Raw Frame
        ROI_frame = resized_frame[94:348, 166:275]
         # ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
        blank_img = np.zeros_like(resized_frame)
        imgContour = blank_img.copy()

        x = 94 #initial column number
        y = 166 #initial row number
        for i in range(0, 254):
          for j in range(0, 109):

            if ROI_frame[i][j] != 0:
              blank_img[x + i, y + j] = ROI_frame[i, j] 

        fgmask = fgbg.apply(blank_img)

        #Applying Basic Filters
        #############################################################
        imgBlur = cv2.GaussianBlur(blank_img,(7,7), 1)
        # erode = cv2.erode(imgBlur, None, iterations=2)
        # dilate = cv2.dilate(erode, None, iterations=2)
        gaussian = 255*imgBlur+100
        # median = cv2.medianBlur(contrast_imgBlur, 7)
        ##############################################################

        #Applying Background Subtraction (detecting regions of motion within frame)
        fgmaskV2 = fgbg.apply(imgBlur)

        maskV2 = cv2.erode(fgmaskV2, None, iterations=2) 
        maskV2 = cv2.dilate(maskV2, None, iterations=2)

        #Applying Contour detection (detectiong only regions past certain size)
        # imgCanny_initial = cv2.Canny(imgBlur, 155, 191)
        getContours(fgmaskV2,imgContour)
        # getContours(maskV2,imgContour)

        #Applying trajectory/velocity tracking (detecting orientation of motion and see how fast region is moving) --> CURRENTLY NOT WORKING
        
        #Overlaying segmentations onto B-mode image
        #############################################################################################
        fgmaskV2_color = cv2.applyColorMap(imgContour, cv2.COLORMAP_INFERNO)
        resized_frame_revert = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2RGB)
        overlay = cv2.addWeighted(resized_frame_revert, 0.5, fgmaskV2_color, 0.5, 1.0)
        # cv2.imshow("Bmode Overlay", overlay)
        ###########################################################################################

        # Debugging Statements
        cv2.imshow('normal frame', resized_frame)
        # cv2.imshow('FGmask', fgmask)
        # cv2.imshow('Basic Filters', imgBlur)
        # cv2.imshow('Canny', imgCanny_initial)
        # cv2.imshow('FGmaskV2', fgmaskV2)
        # cv2.imshow('FGmaskV2_withfilters', maskV2)
        cv2.imshow('Contour', imgContour)
        # cv2.imshow('trajectory', trajectory_frame)
        # print("raw image shape = {}".format(resized_frame.shape))
        # print("segmented image shape = {}".format(fgmaskV2_color.shape))
        # print("raw image shape (reverted)= {}".format(resized_frame_revert.shape))
        
        #Saving comparison frames as gif 
        resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2BGR)
        overlay = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)
        stack = np.hstack((resized_frame, overlay))
        # cv2.imshow("stacked", stack)
        image_lst.append(stack)


        # Press Q on keyboard to  exit
        if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
            break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\video\src\lkpyramid.cpp:1394: error: (-215:Assertion failed) prevPyr[level * lvlStep1].size() == nextPyr[level * lvlStep2].size() in function 'cv::`anonymous-namespace'::SparsePyrLKOpticalFlowImpl::calc'


In [18]:
imageio.mimsave('Outputs/V1_video.gif', image_lst, fps=30)